In [ ]:
from keras.applications import vgg16
img_rows, img_cols = 32, 32
VGG=vgg16.VGG16(weights='imagenet',include_top=False ,input_shape=(img_rows, img_cols,3))

In [ ]:
for layer in VGG.layers:
    layer.trainable=False

In [ ]:
def top(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import batch_normalization
from tensorflow.keras.models import Model

num_classes = 3

FC_Head = top(VGG, num_classes)

model = Model(inputs = VGG.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/MyDrive/MiniProject_Satellite/Data Set Sattelite/Train'
validation_data_dir = '/content/drive/MyDrive/MiniProject_Satellite/Data Set Sattelite/Validation'
 
train_datagen = ImageDataGenerator(rescale=1./255)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 7029 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.


In [ ]:
from keras.optimizers import rmsprop_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping
                  
checkpoint = ModelCheckpoint("satellite2_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop_v2.RMSprop(learning_rate=0.001),
              metrics = ['accuracy'])

nb_train_samples = 7029
nb_validation_samples = 1500
 
epochs = 10
batch_size = 32

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/10
219/219 [==============================] - ETA: 0s - loss: 0.1977 - accuracy: 0.9315
Epoch 00001: val_loss improved from inf to 0.11101, saving model to satellite2_vgg.h5
219/219 [==============================] - 1863s 9s/step - loss: 0.1977 - accuracy: 0.9315 - val_loss: 0.1110 - val_accuracy: 0.9620
Epoch 2/10
219/219 [==============================] - ETA: 0s - loss: 0.0994 - accuracy: 0.9708
Epoch 00002: val_loss improved from 0.11101 to 0.07080, saving model to satellite2_vgg.h5
219/219 [==============================] - 20s 92ms/step - loss: 0.0994 - accuracy: 0.9708 - val_loss: 0.0708 - val_accuracy: 0.9837
Epoch 3/10
219/219 [==============================] - ETA: 0s - loss: 0.0843 - accuracy: 0.9771
Epoch 00003: val_loss improved from 0.07080 to 0.06067, saving model to satellite2_vgg.h5
219/219 [==============================] - 20s 92ms/step - loss: 0.0843 - accuracy: 0.9771 - val_loss: 0.0607 - val_accuracy: 0.9789
Epoch 4/10
219/219 [===========================